TODO here: mess around with this and see if you can get it working, particularly with custom tokenizer.

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

In [2]:
raw_dataset = load_dataset("royal42/lichess_elite_games")

Found cached dataset parquet (C:/Users/jayor/.cache/huggingface/datasets/royal42___parquet/royal42--lichess_elite_games-24250dad91f3120d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("../data/tokenizations/chess-tokenizer/") # Could change this to custom tokenizer

In [5]:
context_length = 128

In [6]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        max_length=context_length,
        truncation=True
    )
    return outputs


tokenized_datasets = raw_dataset.map(
    tokenize, batched=True, remove_columns=raw_dataset["train"].column_names
)
tokenized_datasets

Loading cached processed dataset at C:\Users\jayor\.cache\huggingface\datasets\royal42___parquet\royal42--lichess_elite_games-24250dad91f3120d\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-8c61257eb97a7fca.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2032806
    })
})

In [7]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [8]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [9]:
train_size = 10_000
test_size = int(0.1 * train_size)
split_dataset = tokenized_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)

Setup model

In [10]:
from transformers import GPT2LMHeadModel
model_checkpoint = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_checkpoint)

Set up trainer classes

In [11]:
training_args = TrainingArguments(
    output_dir='./test_trainer',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    evaluation_strategy="steps",
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    fp16=True,
)

In [12]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test']
)

In [13]:
trainer.train()

c:\Users\jayor\anaconda3\envs\chess_transformer\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/625 [00:00<?, ?it/s]

{'loss': 3.0395, 'learning_rate': 4.96e-05, 'epoch': 0.8}


  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 2.4002556800842285, 'eval_runtime': 7.3838, 'eval_samples_per_second': 135.431, 'eval_steps_per_second': 8.532, 'epoch': 0.8}
{'train_runtime': 313.6417, 'train_samples_per_second': 31.884, 'train_steps_per_second': 1.993, 'train_loss': 2.91679833984375, 'epoch': 1.0}


TrainOutput(global_step=625, training_loss=2.91679833984375, metrics={'train_runtime': 313.6417, 'train_samples_per_second': 31.884, 'train_steps_per_second': 1.993, 'train_loss': 2.91679833984375, 'epoch': 1.0})

In [13]:
trainer.save_model()